In [2]:
import torch
import torchvision.transforms as transforms
import flwr as fl
import logging
from model.yolov5.utils import augmentations, loss
from load_data import load_data
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import yaml
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10


In [3]:
from model.init_yolo import init_yolo
from types import SimpleNamespace

Install Weights & Biases for experiment logging via 'pip install wandb' (recommended)
Install Weights & Biases for experiment logging via 'pip install wandb' (recommended)


In [4]:
def fit_config( yaml_path):
    """
    def fit_config(server_uri, client_id, yaml_path):
    Provides fit configuration for the client.
    Args:
        server_uri: The URI of the Flower server.
        client_id: The ID of the client.
        yaml_path: Path to the YAML configuration file.

    Returns:
        dict: A dictionary containing fit configuration.
    """
    # Load YAML configuration
    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)

    # Update config with additional parameters or overrides (optional)
    return config

print(fit_config(r"/home/siu856522160/major/test/object_detection/config/config_yolov5.yaml"))
args = fit_config(r"/home/siu856522160/major/test/object_detection/config/config_yolov5.yaml")
args = SimpleNamespace(**args)

{'yaml_config_file': 'config/fedml_config_yolov5.yaml', 'using_mlops': False, 'yaml_paths': ['config/'], 'training_type': 'cross_silo', 'random_seed': 0, 'scenario': 'horizontal', 'config_version': 'release', 'name': 'exp', 'project': 'runs/train', 'exist_ok': True, 'bootstrap': 'config/bootstrap.sh', 'dataset': 'coco', 'partition_method': 'homo', 'partition_alpha': 0.5, 'data_conf': '/home/siu856522160/major/test/object_detection/model/yolov5/data/coco128.yaml', 'img_size': [640, 640], 'model': 'yolov5', 'class_num': 80, 'yolo_cfg': '/home/siu856522160/major/test/object_detection/model/yolov5/models/yolov5s.yaml', 'yolo_hyp': '/home/siu856522160/major/test/object_detection/config/hyps/hyp.scratch.yaml', 'weights': 'none', 'single_cls': False, 'conf_thres': 0.001, 'iou_thres': 0.6, 'yolo_verbose': False, 'federated_optimizer': 'FedAvg', 'client_num_in_total': 3, 'client_num_per_round': 1, 'comm_round': 20, 'epochs': 5, 'batch_size': 1, 'client_optimizer': 'adam', 'lr': 0.01, 'weight_de

In [5]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [6]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [7]:
# Print key-value pairs
for key, value in vars(args).items():
    print(f"{key}: {value}")

yaml_config_file: config/fedml_config_yolov5.yaml
using_mlops: False
yaml_paths: ['config/']
training_type: cross_silo
random_seed: 0
scenario: horizontal
config_version: release
name: exp
project: runs/train
exist_ok: True
bootstrap: config/bootstrap.sh
dataset: coco
partition_method: homo
partition_alpha: 0.5
data_conf: /home/siu856522160/major/test/object_detection/model/yolov5/data/coco128.yaml
img_size: [640, 640]
model: yolov5
class_num: 80
yolo_cfg: /home/siu856522160/major/test/object_detection/model/yolov5/models/yolov5s.yaml
yolo_hyp: /home/siu856522160/major/test/object_detection/config/hyps/hyp.scratch.yaml
weights: none
single_cls: False
conf_thres: 0.001
iou_thres: 0.6
yolo_verbose: False
federated_optimizer: FedAvg
client_num_in_total: 3
client_num_per_round: 1
comm_round: 20
epochs: 5
batch_size: 1
client_optimizer: adam
lr: 0.01
weight_decay: 0.001
checkpoint_interval: 5
server_checkpoint_interval: 5
frequency_of_the_test: 5
worker_num: 2
using_gpu: True
log_file_dir: 

In [8]:
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model, dataset, trainer, args = init_yolo(args=args, device=DEVICE)

In [9]:
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model, dataset, trainer, args = init_yolo(args=args, device=DEVICE)
# class YOLOFlowerClient(fl.client.NumPyClient):
#     def __init__(self, cid, net, trainloader, valloader):
#         self.cid = cid
#         self.net = net
#         self.trainloader = trainloader
#         self.valloader = valloader

#     def get_parameters(self, config):
#         print(f"[Client {self.cid}] get_parameters")
#         return get_parameters(self.net)

#     def fit(self, parameters, config):
#         print(f"[Client {self.cid}] fit, config: {config}")
#         set_parameters(self.net, parameters)
#         trainer.train(dataset, DEVICE, args)  
#         # train(self.net, self.trainloader, epochs=1)
#         return get_parameters(self.net), len(self.trainloader), {}

#     def evaluate(self, parameters, config):
#         print(f"[Client {self.cid}] evaluate, config: {config}")
#         set_parameters(self.net, parameters)

#         trainer.val(model, dataset, DEVICE, args)



In [10]:
# class YOLOFlowerClient(fl.client.NumPyClient):
#     def __init__(self, cid, net, trainloader, valloader):
#         super().__init__()  # Call parent class constructor for NumPyClient
#         self.cid = cid
#         self.net = net.to(DEVICE)  # Move model to device immediately
#         self.trainloader = trainloader
#         self.valloader = valloader

#     def get_parameters(self, config):
#         print(f"[Client {self.cid}] get_parameters")
#         return self.net.state_dict()  # Use model's state_dict for efficiency

#     def fit(self, parameters, config):
#         print(f"[Client {self.cid}] fit, config: {config}")
#         self.net.load_state_dict(parameters)  # Load parameters for efficiency
#         model.train()  # Set model to training mode
#         with torch.no_grad():  # Faster execution for validation
#             model.val()   # Set model to validation mode
#             self.net(next(iter(self.trainloader))[0].to(DEVICE))  # Forward pass for model loading
#         model.train()  # Restore training mode for FL training
#         return self.net.state_dict(), len(self.trainloader), {}

#     def evaluate(self, parameters, config):
#         print(f"[Client {self.cid}] evaluate, config: {config}")
#         self.net.load_state_dict(parameters)
#         with torch.no_grad():
#             self.net.eval()  # Set model to evaluation mode
#             model(next(iter(self.valloader))[0].to(DEVICE))  # Forward pass for evaluation



In [11]:
# def numpyclient_fn(cid) -> FlowerNumPyClient:
#     net = Net().to(DEVICE)
#     trainloader = trainloaders[int(cid)]
#     valloader = valloaders[int(cid)]
#     return FlowerNumPyClient(cid, net, trainloader, valloader)


In [12]:
# def fit(config, client_id):
#     """Trains the YOLOv5 model on the client's data."""

#     # Load data
#     images, labels = load_data(client_id)

#     # Load or create YOLOv5 model
#     model = YOLOv5s(**config["model_params"])
#     model.to(device="cpu")  # Ensure model is on CPU for client training

#     # Define optimizer and loss function
#     optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"])
#     loss_fn = loss.ComputeLoss(**config["loss_params"])

#     # Set up logging
#     logger = logging.getLogger("client_" + str(client_id))

#     # Training loop
#     for epoch in range(config["epochs"]):
#         for image, label in zip(images, labels):
#             # Augment image (optional, adjust based on your needs)
#             image, label = augmentations.augment_hsv(image, label)
#             image, label = augmentations.letterbox(image, label, 640, auto=True, scaleup=True)

#             # Convert to PyTorch tensors
#             image = torch.from_numpy(image).float() / 255.0
#             image = image.permute(2, 0, 1).unsqueeze(0)  # Add batch dimension
#             label = torch.from_numpy(label)

#             # Forward pass & calculate loss
#             with torch.no_grad():
#                 preds = model(image)
#             loss = loss_fn(preds, label)

#             # Backpropagation & update weights
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # Log training progress
#             logger.info(f"Epoch {epoch+1}/{config['epochs']}, Loss: {loss.item()}")

#     # Get updated model and metrics
#     updated_model = model
#     metrics = {"loss": loss.item()}  # Calculate other metrics as needed

#     return updated_model, metrics




In [13]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, dataset, dataset_train, dataset_val, trainer, args = init_yolo(args=args, device=DEVICE)

<class 'types.SimpleNamespace'>


add checkpoint interval
add checkpoint interval
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0}
Hyperparameters {'lr0': 0.01, 'lrf': 0.2, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0}
add file handler
add file handler

         

  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  models.common.C3                        [512, 512, 1]                 
  8                -1  1   1182720  m


/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/images/train2017
/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/images/train2017
/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/images/train2017


Scanning '/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017' images and labels...126 found, 2 missing, 0 empty, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 12296.91it/s]
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache
Scanning '/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017' images and labels...126 found, 2 missing, 0 empty, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 11484.12it/s]
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache


/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/images/train2017


Scanning '/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017' images and labels...126 found, 2 missing, 0 empty, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 12250.89it/s]
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache


/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128 None
/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/images/train2017


Scanning '/home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017' images and labels...126 found, 2 missing, 0 empty, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 11997.92it/s]
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache
New cache created: /home/siu856522160/major/test/object_detection/model/yolov5/datasets/coco128/labels/train2017.cache


In [14]:
# print(f"model: {model}")
print(f"dataset: {dataset}")
print(f"dataset: {dataset_train}")
print(f"dataset: {dataset_val}")
print(f"trainer: {trainer}")
print(f"args: {args}")

dataset: (128, 128, <model.yolov5.utils.dataloaders.InfiniteDataLoader object at 0x7fac984eeb00>, <model.yolov5.utils.dataloaders.InfiniteDataLoader object at 0x7fab903b7220>, {}, {}, {}, 80)
dataset: (128, <model.yolov5.utils.dataloaders.InfiniteDataLoader object at 0x7fab905cca90>, (80, ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard

In [15]:
# import flwr as fl

# fl.server.start_server(config=fl.server.ServerConfig(num_rounds=3))

INFO flwr 2024-02-26 13:40:53,646 | app.py:163 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2024-02-26 13:40:53,651 | app.py:176 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
Flower ECE: gRPC server running (3 rounds), SSL is disabled
Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flwr 2024-02-26 13:40:53,658 | server.py:89 | Initializing global parameters
Initializing global parameters
Initializing global parameters
INFO flwr 2024-02-26 13:40:53,659 | server.py:276 | Requesting initial parameters from one random client
Requesting initial parameters from one random client
Requesting initial parameters from one random client


KeyboardInterrupt: 

In [20]:
import torch  # Assuming torch for model
import flwr  # Assuming flwr for federated learning
import logging
from flwr.server import ClientManager

from trainer.yolov5_trainer import YOLOv5Trainer

def main():

    # Start Flower server
    fl.client.start_client(
        server_address="0.0.0.0:8080",
        # config=fl.server.ServerConfig(num_rounds=3),
        client_fn= trainer
        )

if __name__ == "__main__":
    main()


INFO flwr 2024-02-26 15:06:01,059 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
Opened insecure gRPC connection (no certificates were passed)
Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-02-26 15:06:01,066 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-02-26 15:06:01,068 | connection.py:55 | ChannelConnectivity.READY


Exception iterating responses: 
Traceback (most recent call last):
  File "/home/siu856522160/anaconda3/envs/flower/lib/python3.10/site-packages/grpc/_server.py", line 599, in _take_response_from_response_iterator
    return next(response_iterator), True
  File "/home/siu856522160/anaconda3/envs/flower/lib/python3.10/site-packages/flwr/server/fleet/grpc_bidi/flower_service_servicer.py", line 113, in Join
    ins_wrapper: InsWrapper = next(ins_wrapper_iterator)
  File "/home/siu856522160/anaconda3/envs/flower/lib/python3.10/site-packages/flwr/server/fleet/grpc_bidi/grpc_bridge.py", line 165, in ins_wrapper_iterator
    self._raise_if_closed()
  File "/home/siu856522160/anaconda3/envs/flower/lib/python3.10/site-packages/flwr/server/fleet/grpc_bidi/grpc_bridge.py", line 81, in _raise_if_closed
    raise GrpcBridgeClosed()
flwr.server.fleet.grpc_bidi.grpc_bridge.GrpcBridgeClosed
Exception iterating responses: 
Traceback (most recent call last):
  File "/home/siu856522160/anaconda3/envs/flo

DEBUG flwr 2024-02-26 15:06:19,676 | connection.py:220 | gRPC channel closed


KeyboardInterrupt: 

In [ ]:
# # Start simulation
# fl.simulation.start_simulation(
#     client_fn=client_fn,
#     num_clients=2,
#     config=fl.server.ServerConfig(num_rounds=5),
#     # strategy=strategy,
#     # client_resources=client_resources,
# )